In [418]:
import math as m
import numpy as np
import sys
import sympy as sym

# General Calculations

In [467]:
# Get shear modulus from elastic modulus
def shear_mod(E, v):
    return E/(2*(1+v))

In [466]:
# Get elastic modulus from shear modulus
def elastic_mod(E, v):
    return G*(2*(1+v))

### 2D Eqns 1.18a, b, c - Transformation Equations for Plane Stress

In [15]:
def plane_stress_trans(sigma_x, sigma_y, tau_xy, theta):
    sigma_x_new = 0.5*(sigma_x + sigma_y) + 0.5*(sigma_x - sigma_y)*m.cos(2*theta*m.pi/180) + tau_xy*m.sin(2*theta*m.pi/180)
    sigma_y_new = 0.5*(sigma_x + sigma_y) - 0.5*(sigma_x - sigma_y)*m.cos(2*theta*m.pi/180) - tau_xy*m.sin(2*theta*m.pi/180)
    tau_xy_prime = -0.5*(sigma_x - sigma_y)*m.sin(2*theta*m.pi/180) + tau_xy*m.cos(2*theta*m.pi/180)
    return sigma_x_new, sigma_y_new, tau_xy_prime

Enter values (in MPa, degrees)

In [18]:
sigma_x = 45
sigma_y = 90
tau_xy = 0
theta = 53.13
sigma_x_new, sigma_y_new, tau_xy_prime = plane_stress_trans(sigma_x, sigma_y, tau_xy, theta)
print('sigma_x_prime = %.2f MPa' % sigma_x_new)
print('sigma_y_prime = %.2f MPa' % sigma_y_new)
print('tau_xy_prime = %.2f MPa' % tau_xy_prime)

sigma_x_prime = 73.80 MPa
sigma_y_prime = 61.20 MPa
tau_xy_prime = 21.60 MPa


### Mohr's circle calculator

https://www.engineeringcalculator.net/mohrs_circle.html

### 2D Principal stresses

In [46]:
def principal_stress(sigma_x, sigma_y, tau_xy):
    sigma_max = (sigma_x+sigma_y)/2 + m.sqrt(((sigma_x-sigma_y)/2)**2 + tau_xy**2)
    sigma_min = (sigma_x+sigma_y)/2 - m.sqrt(((sigma_x-sigma_y)/2)**2 + tau_xy**2)
    return sigma_max, sigma_min

In [480]:
sigma_x = 100
sigma_y = 40
tau_xy = 40
sigma_max, sigma_min = principal_stress(sigma_x, sigma_y, tau_xy)
print('sigma_max = %.2f MPa' % sigma_max)
print('sigma_min = %.2f MPa' % sigma_min)

sigma_max = 120.00 MPa
sigma_min = 20.00 MPa


### 2D Max shearing stress

In [484]:
def max_shear_stress(sigma_x, sigma_y, tau_xy):
    tau_max_1 = m.sqrt(((sigma_x-sigma_y)/2)**2 + tau_xy**2)
    tau_max_2 = -m.sqrt(((sigma_x-sigma_y)/2)**2 + tau_xy**2)
    return tau_max_1, tau_max_2

In [485]:
sigma_x = 100
sigma_y = 40
tau_xy = 40
tau_1, tau_2 = max_shear_stress(sigma_x, sigma_y, tau_xy)
print('tau_max = %.2f MPa' % tau_2)

tau_max = -50.00 MPa


### 2D Normal stress acting on planes of max shearing stress 
Reference Eqn. 1.23

In [21]:
def sigma_prime(sigma_x, sigma_y):
    sigma_prime = 0.5*(sigma_x + sigma_y)
    return sigma_prime

Enter values (in MPa)

In [69]:
sigma_x = 186
sigma_y = -12
print('Normal stress on max shearing plane: %.2f MPa' % sigma_prime(sigma_x, sigma_y))

Normal stress on max shearing plane: 87.00 MPa


### 2D Principal and shear plane directions (angle of rotation for normal stresses/principal stresses)

In [62]:
def theta_ps(sigma_x, sigma_y, tau_xy):
    theta_p = (180/m.pi)*m.atan((2*tau_xy/(sigma_x-sigma_y)))/2
    theta_s = (180/m.pi)*m.atan((-(sigma_x-sigma_y)/(2*tau_xy)))/2
    theta_s = m.atan(-(sigma_x-sigma_y)/(2*tau_xy))/2
    return theta_p, theta_s

In [482]:
sigma_x = 100
sigma_y = 40
tau_xy = 40
theta_p, theta_s = theta_ps(sigma_x, sigma_y, tau_xy)
print('theta_p: %.2f deg' % theta_p)
print('theta_s: %.2f deg' % theta_s)

theta_p: 26.57 deg
theta_s: -18.43 deg


# 3D Calculations

### 3D Principal Stresses (x, y, z coordinate system only)

In [504]:
sigma_x = 50
tau_xy = 40
tau_xz = 0
sigma_y = -30
tau_yz = 0
sigma_z = 20

# Calculate principal stresses
stress_matrix = np.array([[sigma_x, tau_xy, tau_xz],
                         [tau_xy, sigma_y, tau_yz],
                         [tau_xz, tau_yz, sigma_z]])
i1 = sigma_x + sigma_y + sigma_z
i2 = sigma_x*sigma_y + sigma_y*sigma_z + sigma_x*sigma_z - tau_xy**2 - tau_yz**2 - tau_xz**2
i3 = np.linalg.det(stress_matrix)
sigma_p = sorted(np.roots([1, -i1, i2, -i3]), reverse=True)

# Calculate directional cosines (l, m, n)
a = np.linalg.det(
    np.array([[sigma_y-sigma_p[0], tau_yz],
              [tau_yz, sigma_z-sigma_p[0]]]))
b = -np.linalg.det(
    np.array([[tau_xy, tau_yz],
              [tau_xz, sigma_z-sigma_p[0]]]))
c = np.linalg.det(
    np.array([[tau_xy, sigma_y-sigma_p[0]],
              [tau_xz, tau_yz]]))
k = 1/m.sqrt(a**2 + b**2 + c**2)
l1 = a*k
m1 = b*k
n1 = c*k
dir_cos = [l1, m1, n1]

sigma_oct = sum(sigma_p)/3
tau_oct = m.sqrt((sigma_p[0]-sigma_p[1])**2 + (sigma_p[1]-sigma_p[2])**2 + (sigma_p[2]-sigma_p[0])**2)/3
tau_max = (sigma_p[0]-sigma_p[2])/2

# print('Prime CSYS stresses: ', prime_stress_matrix)
print('Invariants: ', i1, i2, i3)
print('Principal stresses: ', sigma_p)
print('Direction: ', dir_cos)
print('Octahedral normal stress: %.2f MPa' % sigma_oct)
print('Octahedral shear stress: %.2f MPa' % tau_oct)
print('Abs max shearing stress: %.2f MPa' % tau_max)

Invariants:  40 -2700 -61999.999999999935
Principal stresses:  [66.56854249492379, 19.999999999999982, -46.568542494923804]
Direction:  [0.9238795325112866, 0.38268343236509, 0.0]
Octahedral normal stress: 13.33 MPa
Octahedral shear stress: 46.43 MPa
Abs max shearing stress: 56.57 MPa


### 3D Principal Stresses with CSYS Rotation

In [503]:
dir_cos_matrix = angle2dcm(45, 0, 0)
[l1, m1, n1], [l2, m2, n2], [l3, m3, n3] = dir_cos_matrix
# Override for oblique planes

sigma_x = 100
tau_xy = 100
tau_xz = 0
sigma_y = 100
tau_yz = 0
sigma_z = 100

sigma_x_p = sigma_x*(l1**2) + sigma_y*(m1**2) + sigma_z*(n1**2) + 2*(tau_xy*l1*m1 + tau_yz*m1*n1 + tau_xz*l1*n1)
sigma_y_p = sigma_x*(l2**2) + sigma_y*(m2**2) + sigma_z*(n2**2) + 2*(tau_xy*l2*m2 + tau_yz*m2*n2 + tau_xz*l2*n2)
sigma_z_p = sigma_x*(l3**2) + sigma_y*(m3**2) + sigma_z*(n3**2) + 2*(tau_xy*l3*m3 + tau_yz*m3*n3 + tau_xz*l3*n3)
tau_xy_p = sigma_x*l1*l2 + sigma_y*m1*m2 + sigma_z*n1*n2 + tau_xy*(l1*m2+m1*l2) + tau_yz*(m1*n2+n1*m2) + tau_xz*(n1*l2+l1*n2)
tau_xz_p = sigma_x*l1*l3 + sigma_y*m1*m3 + sigma_z*n1*n3 + tau_xy*(l1*m3+m1*l3) + tau_yz*(m1*n3+n1*m3) + tau_xz*(n1*l3+l1*n3)
tau_yz_p = sigma_x*l2*l3 + sigma_y*m2*m3 + sigma_z*n2*n3 + tau_xy*(l2*m3+m2*l3) + tau_yz*(m2*n3+n2*m3) + tau_xz*(n2*l3+l2*n3)

stress_matrix_p = np.array([[sigma_x_p, tau_xy_p, tau_xz_p],
                         [tau_xy_p, sigma_y_p, tau_yz_p],
                         [tau_xz_p, tau_yz_p, sigma_z_p]])

i1 = sigma_x_p + sigma_y_p + sigma_z_p
i2 = sigma_x_p*sigma_y_p + sigma_y_p*sigma_z_p + sigma_x_p*sigma_z_p - tau_xy_p**2 - tau_yz_p**2 - tau_xz_p**2
i3 = np.linalg.det(stress_matrix_p)
sigma_p = sorted(np.roots([1, -i1, i2, -i3]), reverse=True)

sigma_oct = sum(sigma_p)/3
tau_oct = m.sqrt((sigma_p[0]-sigma_p[1])**2 + (sigma_p[1]-sigma_p[2])**2 + (sigma_p[2]-sigma_p[0])**2)/3
tau_max = (sigma_p[0]-sigma_p[2])/2

p = []
for i in sigma_p:
    p.append(i/m.sqrt(3))
    
print(np.linalg.norm(p))

# print(dir_cos_matrix)

print(stress_matrix_p)
print('Sigma_oct: ', sigma_oct)
print(sigma_p)
print('Abs max shearing stress: %.2f MPa' % tau_max)

129.0994448735807
[[2.00000000e+02 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.42108547e-14 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.00000000e+02]]
Sigma_oct:  100.00000000000007
[200.00000000000023, 100.0, 1.4210854715201982e-14]
Abs max shearing stress: 100.00 MPa


In [490]:
shear_mod(200e9, 0.3)*1e-9

76.92307692307692

### Normal and Shear Stresses on an Oblique Plane

In [350]:
dir_cos_matrix = angle2dcm(-53.13, 0, 56.3)
[l1, m1, n1], [l2, m2, n2], [l3, m3, n3] = dir_cos_matrix
# Override for oblique planes

sigma_x = 100
tau_xy = 40
tau_xz = 0
sigma_y = 60
tau_yz = 80
sigma_z = 20

sigma_x_p = sigma_x*(l1**2) + sigma_y*(m1**2) + sigma_z*(n1**2) + 2*(tau_xy*l1*m1 + tau_yz*m1*n1 + tau_xz*l1*n1)
sigma_y_p = sigma_x*(l2**2) + sigma_y*(m2**2) + sigma_z*(n2**2) + 2*(tau_xy*l2*m2 + tau_yz*m2*n2 + tau_xz*l2*n2)
sigma_z_p = sigma_x*(l3**2) + sigma_y*(m3**2) + sigma_z*(n3**2) + 2*(tau_xy*l3*m3 + tau_yz*m3*n3 + tau_xz*l3*n3)
tau_xy_p = sigma_x*l1*l2 + sigma_y*m1*m2 + sigma_z*n1*n2 + tau_xy*(l1*m2+m1*l2) + tau_yz*(m1*n2+n1*m2) + tau_xz*(n1*l2+l1*n2)
tau_xz_p = sigma_x*l1*l3 + sigma_y*m1*m3 + sigma_z*n1*n3 + tau_xy*(l1*m3+m1*l3) + tau_yz*(m1*n3+n1*m3) + tau_xz*(n1*l3+l1*n3)
tau_yz_p = sigma_x*l2*l3 + sigma_y*m2*m3 + sigma_z*n2*n3 + tau_xy*(l2*m3+m2*l3) + tau_yz*(m2*n3+n2*m3) + tau_xz*(n2*l3+l2*n3)

stress_matrix_p = np.array([[sigma_x_p, tau_xy_p, tau_xz_p],
                         [tau_xy_p, sigma_y_p, tau_yz_p],
                         [tau_xz_p, tau_yz_p, sigma_z_p]])

i1 = sigma_x_p + sigma_y_p + sigma_z_p
i2 = sigma_x_p*sigma_y_p + sigma_y_p*sigma_z_p + sigma_x_p*sigma_z_p - tau_xy_p**2 - tau_yz_p**2 - tau_xz_p**2
i3 = np.linalg.det(stress_matrix_p)
sigma_p = sorted(np.roots([1, -i1, i2, -i3]), reverse=True)

sigma_oct = sum(sigma_p)/3
tau_oct = m.sqrt((sigma_p[0]-sigma_p[1])**2 + (sigma_p[1]-sigma_p[2])**2 + (sigma_p[2]-sigma_p[0])**2)/3
tau_max = (sigma_p[0]-sigma_p[2])/2

print(dir_cos_matrix)
sigma_normal = sigma_x*(l1**2) + sigma_y*(m1**2) + sigma_z*n**2 + 2*(tau_xy*l1*m1 + tau_yz*m1*n1 + tau_xz*l1*n1)

# print(dir_cos_matrix)

print(stress_matrix_p)
print(i1, i2, i3)
print('Normal stress: ', sigma_normal)
print('Abs max shearing stress: %.2f MPa' % tau_max)

[[ 0.60000143 -0.79999893  0.        ]
 [ 0.44387495  0.33290745  0.83195412]
 [-0.66556241 -0.49917366  0.55484443]]
[[ 36.00002858 -48.80614983 -42.16575954]
 [-48.80614983  96.33083039 -66.45313752]
 [-42.16575954 -66.45313752  47.66914102]]
180.00000000000003 1200.0000000000014 -552000.0000000007
Normal stress:  36.00002858295324
Abs max shearing stress: 96.28 MPa


# Strain Calculations

### Change of length between two points with given strain
c = constant (typically 10e-6)

In [276]:
pt1 = [0, 0]
pt2 = [0, 0.06]
eps_x = 0
eps_y = 0
gamma_xy = 0.000910

d = m.dist(pt1, pt2)
theta = m.atan((pt2[1]-pt1[1])/(pt2[0]-pt1[0]))*180/m.pi

e_12 = (eps_x+eps_y)/2 + (eps_x-eps_y)*m.cos(2*theta*m.pi/180)/2 + gamma_xy*m.sin(2*theta*m.pi/180)/2
d2 = d*e_12
print('Change of length (m): ', d2)

Change of length (m):  0.0


### 3D Volume Change of Block

In [390]:
E = 210e9
v = 0.3
w = 2
h = 1.5
d = -1

# If stresses provided
sigma_x = -160e6
sigma_y = -160e6
sigma_z = -160e6

eps_x = (1/E)*(sigma_x - v*(sigma_y + sigma_z))
eps_y = (1/E)*(sigma_y - v*(sigma_x + sigma_z))
eps_z = (1/E)*(sigma_z - v*(sigma_x + sigma_y))
dx = eps_x * w
dy = eps_y * h
dz = eps_z * d

eps_total = eps_x + eps_y + eps_z

dV = eps_total*w*h*d

print('Strains: ', eps_x, eps_y, eps_z)
print('Displacements: ', dx, dy, dz)
print('dV: ', dV)

Strains:  -0.00030476190476190474 -0.00030476190476190474 -0.00030476190476190474
Displacements:  -0.0006095238095238095 -0.00045714285714285713 0.00030476190476190474
dV:  0.002742857142857143


### 3D Strain Calculator - Given CSYS Rotation

In [479]:
pt1 = [3, 1, 2]
eps_x = 347
gamma_xy = 0
gamma_xz = 0
eps_y = -104
gamma_yz = 0
eps_z = 0

# If points are provided to calculate vectors and cosines with
pt1 = [0, 0, 0]
pt2 = [3, 2, 0]
pt3 = [0, 0, 0]
# l1 = (pt2[0]-pt1[0])/m.sqrt((pt2[0]-pt1[0])**2 + (pt2[1]-pt1[1])**2 + (pt2[2]-pt1[2])**2)
# m1 = (pt2[1]-pt1[1])/m.sqrt((pt2[0]-pt1[0])**2 + (pt2[1]-pt1[1])**2 + (pt2[2]-pt1[2])**2)
# n1 = (pt2[2]-pt1[2])/m.sqrt((pt2[0]-pt1[0])**2 + (pt2[1]-pt1[1])**2 + (pt2[2]-pt1[2])**2)
# l2 = (pt3[0]-pt1[0])/m.sqrt((pt3[0]-pt1[0])**2 + (pt3[1]-pt1[1])**2 + (pt3[2]-pt1[2])**2)
# m2 = (pt3[1]-pt1[1])/m.sqrt((pt3[0]-pt1[0])**2 + (pt3[1]-pt1[1])**2 + (pt3[2]-pt1[2])**2)
# n2 = (pt3[2]-pt1[2])/m.sqrt((pt3[0]-pt1[0])**2 + (pt3[1]-pt1[1])**2 + (pt3[2]-pt1[2])**2)

# Override if angle provided
# Angle in ZYX
dir_cos_matrix = angle2dcm(-30, 0, 0)
[l1, m1, n1], [l2, m2, n2], [l3, m3, n3] = dir_cos_matrix
print(dir_cos_matrix)

strain_matrix = np.array([[eps_x, gamma_xy/2, gamma_xz/2],
                         [gamma_xy/2, eps_y, gamma_yz/2],
                         [gamma_xz/2, gamma_yz/2, eps_z]])

eps_x_p = eps_x*(l1**2) + eps_y*(m1**2) + eps_z*(n1**2) + gamma_xy*l1*m1 + gamma_yz*m1*n1 + gamma_xz*l1*n1
eps_y_p = eps_x*(l2**2) + eps_y*(m2**2) + eps_z*(n2**2) + gamma_xy*l2*m2 + gamma_yz*m2*n2 + gamma_xz*l2*n2
eps_z_p = eps_x*(l3**2) + eps_y*(m3**2) + eps_z*(n3**2) + gamma_xy*l3*m3 + gamma_yz*m3*n3 + gamma_xz*l3*n3
gamma_xy_p = 2*(eps_x*l1*l2 + eps_y*m1*m2 + eps_z*n1*n2) + gamma_xy*(l1*m2 + m1*l2) + gamma_yz*(m1*n2 + n1*m2) + gamma_xz*(n1*l2 + l1*n2)
gamma_xz_p = 2*(eps_x*l1*l3 + eps_y*m1*m3 + eps_z*n1*n3) + gamma_xy*(l1*m3 + m1*l3) + gamma_yz*(m1*n3 + n1*m3) + gamma_xz*(n1*l3 + l1*n3)
gamma_yz_p = 2*(eps_x*l2*l3 + eps_y*m2*m3 + eps_z*n2*n3) + gamma_xy*(l2*m3 + m2*l3) + gamma_yz*(m2*n3 + n2*m3) + gamma_xz*(n2*l3 + l2*n3)

# Calculate state of transformed strain
strain_matrix_p = np.array([[eps_x_p, gamma_xy_p/2, gamma_xz_p/2],
                         [gamma_xy_p/2, eps_y_p, gamma_yz_p/2],
                         [gamma_xz_p/2, gamma_yz_p/2, eps_z_p]])
j1 = eps_x + eps_y + eps_z
j2 = eps_x*eps_y + eps_y*eps_z + eps_x*eps_z - (gamma_xy**2 + gamma_yz**2 + gamma_xz**2)/4
j3 = np.linalg.det(strain_matrix)
eps_p = sorted(np.roots([1, -j1, j2, -j3]), reverse=True)
gamma_max = [(eps_p[0]-eps_p[2])]

theta_p = 0.5*m.atan(gamma_xy/(eps_x-eps_y))*180/m.pi

# print('Prime CSYS stresses: ', prime_stress_matrix)
print('Strain invariants: ', j1, j2, j3)
print('Principal strains: ', eps_p)
print('Principal strain direction: ', theta_p)
print('Max shear strain: ', gamma_max)
print('Transformed strain matrix: \n', strain_matrix_p)

[[ 0.8660254 -0.5        0.       ]
 [ 0.5        0.8660254  0.       ]
 [ 0.         0.         1.       ]]
Strain invariants:  243 -36088.0 0.0
Principal strains:  [347.0, 0.0, -104.0]
Principal strain direction:  0.0
Max shear strain:  [451.0]
Transformed strain matrix: 
 [[234.25       195.28872855   0.        ]
 [195.28872855   8.75         0.        ]
 [  0.           0.           0.        ]]


### Strain tensor for a state of stress
Input: stress tensor with some material properties

Output: strain tensor

In [310]:
stress_matrix = 1e6*np.array([[34, 48, 8],
                         [48, 5, 1],
                         [8, 1, 5]])
[sigma_x, tau_xy, tau_xz] = stress_matrix[0]
[tau_xy, sigma_y, tau_yz] = stress_matrix[1]
[tau_xz, tau_yz, sigma_z] = stress_matrix[2]

E = 70e9
# G = 80e9
# v = E/(2*G)-1
v = 0.3
G = E/(2*(1+v))

eps_x = (1/E)*(sigma_x-v*(sigma_y+sigma_z))
eps_y = (1/E)*(sigma_y-v*(sigma_x+sigma_z))
eps_z = (1/E)*(sigma_z-v*(sigma_x+sigma_y))
gamma_xy = tau_xy/G
gamma_xz = tau_xz/G
gamma_yz = tau_yz/G

strain_matrix = 1e6*np.array([[eps_x, gamma_xy/2, gamma_xz/2],
                         [gamma_xy/2, eps_y, gamma_yz/2],
                         [gamma_xz/2, gamma_yz/2, eps_z]])
print('Strain tensor (mu): \n', strain_matrix)

Strain tensor (mu): 
 [[442.85714286 891.42857143 148.57142857]
 [891.42857143 -95.71428571  18.57142857]
 [148.57142857  18.57142857 -95.71428571]]


### Stress tensor for a state of strain
Input: strain tensor with some material properties

Output: stress tensor

In [359]:
strain_matrix = 1e-6*np.array([[81.25, -25, 31.25],
                         [-25, -43.75, 62.5],
                         [31.25, 62.5, 50]])
strain_matrix[0][1] = strain_matrix[0][1]*2
strain_matrix[0][2] = strain_matrix[0][2]*2
strain_matrix[1][2] = strain_matrix[1][2]*2
strain_matrix[1][0] = strain_matrix[1][0]*2
strain_matrix[2][0] = strain_matrix[2][0]*2
strain_matrix[2][1] = strain_matrix[2][1]*2

[eps_x, gamma_xy, gamma_xz] = strain_matrix[0]
[gamma_xy, eps_y, gamma_yz] = strain_matrix[1]
[gamma_xz, gamma_yz, eps_z] = strain_matrix[2]

E = 200e9
G = 80e9
v = E/(2*G)-1
lamb = v*E/((1+v)*(1-2*v))
e = eps_x + eps_y + eps_z

sigma_x = 2*G*eps_x + lamb*e
sigma_y = 2*G*eps_y + lamb*e
sigma_z = 2*G*eps_z + lamb*e
tau_xy = G*gamma_xy
tau_xz = G*gamma_xz
tau_yz = G*gamma_yz

stress_matrix = 1e-6*np.array([[sigma_x, tau_xy, tau_xz],
                         [tau_xy, sigma_y, tau_yz],
                         [tau_xz, tau_yz, sigma_z]])
print('Stress tensor (MPa): \n', stress_matrix)

Stress tensor (MPa): 
 [[ 2.00000000e+01 -4.00000000e+00  5.00000000e+00]
 [-4.00000000e+00 -9.31322575e-16  1.00000000e+01]
 [ 5.00000000e+00  1.00000000e+01  1.50000000e+01]]


### Typical Strain Rosette (45 or 60 deg)
Given rosette strains a, b, c

In [384]:
theta = 60 # Rectangular. Delta is 60 deg

eps_a = -800e-6
eps_b = -1000e-6
eps_c = 400e-6
E = 200e9
v = 0.3

if theta == 45:
    eps_principal = sorted([0.5*(eps_a + eps_c + m.sqrt((eps_a-eps_c)**2 + (2*eps_b-eps_a-eps_c)**2)),
                           0.5*(eps_a + eps_c - m.sqrt((eps_a-eps_c)**2 + (2*eps_b-eps_a-eps_c)**2))], reverse=True)
    sigma_principal = sorted([(E/2)*((eps_a+eps_c)/(1-v) + m.sqrt((eps_a-eps_c)**2 + (2*eps_b-eps_a-eps_c)**2)/(1+v)),
                       (E/2)*((eps_a+eps_c)/(1-v) - m.sqrt((eps_a-eps_c)**2 + (2*eps_b-eps_a-eps_c)**2)/(1+v))], reverse=True)
    theta_p = (180/m.pi)*m.atan((2*eps_b - eps_a - eps_c)/(eps_a - eps_c))/2
elif theta == 60:
    eps_principal = sorted([(1/3)*(eps_a + eps_b + eps_c + m.sqrt(2)*m.sqrt((eps_a-eps_b)**2 + (eps_b-eps_c)**2 + (eps_c-eps_a)**2)),
                           (1/3)*(eps_a + eps_b + eps_c - m.sqrt(2)*m.sqrt((eps_a-eps_b)**2 + (eps_b-eps_c)**2 + (eps_c-eps_a)**2))], reverse=True)
    sigma_principal = sorted([(E/3)*((eps_a+eps_b+eps_c)/(1-v) + m.sqrt(2)*m.sqrt((eps_a-eps_b)**2 + (eps_b-eps_c)**2 + (eps_c-eps_a)**2)/(1+v)),
                       (E/3)*((eps_a+eps_b+eps_c)/(1-v) + m.sqrt(2)*m.sqrt((eps_a-eps_b)**2 + (eps_b-eps_c)**2 + (eps_c-eps_a)**2)/(1+v))], reverse=True)
    theta_p = (180/m.pi)*m.atan(m.sqrt(3)*(eps_b - eps_c)/(2*eps_b - eps_a - eps_c))/2
    

print('Principal strains (mu): ', eps_principal)
print('Principal stresses (MPa): ', sigma_principal)
print('Direction of principal plane: ', theta_p)

Principal strains (mu):  [0.00040765846990693344, -0.0013409918032402668]
Principal stresses (MPa):  [1178226.1395282268, 1178226.1395282268]
Direction of principal plane:  28.29100945404395


### General Strain Rosette Solver - assume eps_a, b, c are given

In [469]:
theta_a = -30
theta_b = 15
theta_c = 60
eps_a = -800e-6
eps_b = -1000e-6
eps_c = 400e-6
E = 200e9
v = 0.3
# eps_x = 1
# eps_y = 1
eps_z = 0
# gamma_xy = 1

eps_dir = np.array([eps_a, eps_b, eps_c])
eps_matrix = np.array([[(m.cos(theta_a*m.pi/180))**2, (m.sin(theta_a*m.pi/180))**2, (m.sin(theta_a*m.pi/180)*m.cos(theta_a*m.pi/180))],
                      [(m.cos(theta_b*m.pi/180))**2, (m.sin(theta_b*m.pi/180))**2, (m.sin(theta_b*m.pi/180)*m.cos(theta_b*m.pi/180))],
                      [(m.cos(theta_c*m.pi/180))**2, (m.sin(theta_c*m.pi/180))**2, (m.sin(theta_c*m.pi/180)*m.cos(theta_c*m.pi/180))]])
eps_x, eps_y, gamma_xy = np.linalg.solve(eps_matrix, eps_dir)
eps_3 = (-v/(1-v))*(eps_x + eps_y)

eps_principal = sorted([(eps_x+eps_y)/2 + m.sqrt(((eps_x-eps_y)/2)**2 + (gamma_xy/2)**2),
                       (eps_x+eps_y)/2 - m.sqrt(((eps_x-eps_y)/2)**2 + (gamma_xy/2)**2)], reverse=True)
gamma_max = sorted([(eps_principal[0] - eps_principal[1]), -(eps_principal[0] - eps_principal[1])], reverse=True)
true_gamma_max = sorted([(eps_principal[0] - eps_3), -(eps_principal[0] - eps_3)], reverse=True)
theta_p = m.atan(gamma_xy/(eps_x-eps_y))*180/(m.pi*2)

print('Strains eps_x, eps_y, gamma_xy: ', eps_x, eps_y, gamma_xy)
print('Principal strains: ', eps_principal)
print('Max shear strain: ', gamma_max)
print('True max shear strain: ', true_gamma_max)
print('Principal strain plane angle: ', theta_p)

Strains eps_x, eps_y, gamma_xy:  -0.0008 0.00040000000000000013 -0.0015999999999999996
Principal strains:  [0.0007999999999999998, -0.0011999999999999997]
Max shear strain:  [0.0019999999999999996, -0.0019999999999999996]
True max shear strain:  [0.0006285714285714284, -0.0006285714285714284]
Principal strain plane angle:  26.565051177077986


### Deviatoric and dilatational stresses, given a stress tensor

In [496]:
stress_matrix = 1e6*np.array([[100, 100, 0],
                         [100, 100, 0],
                         [0, 0, 1000]])
[sigma_x, tau_xy, tau_xz] = stress_matrix[0]
[tau_xy, sigma_y, tau_yz] = stress_matrix[1]
[tau_xz, tau_yz, sigma_z] = stress_matrix[2]

# Dilatational stress, sigma_m
sigma_m = (sigma_x + sigma_y + sigma_z)/3
dilatational_matrix = np.identity(3)*sigma_m

# Deviator stress, sigma_m
sigma_xm, sigma_ym, sigma_zm = [sigma_x-sigma_m, sigma_y-sigma_m, sigma_z-sigma_m]
deviator_matrix = stress_matrix
deviator_matrix[0][0] = sigma_xm
deviator_matrix[1][1] = sigma_ym
deviator_matrix[2][2] = sigma_zm

i1 = sigma_xm + sigma_ym + sigma_zm
i2 = sigma_xm*sigma_ym + sigma_ym*sigma_zm + sigma_xm*sigma_zm - tau_xy**2 - tau_yz**2 - tau_xz**2
i3 = np.linalg.det(deviator_matrix)
sigma_m_p = sorted(np.roots([1, -i1, i2, -i3]), reverse=True)

print('Principal deviator stresses (MPa): ', sigma_m_p)
print('Dilatational matrix (MPa): \n', 1e-6*dilatational_matrix)
print('Deviator matrix (MPa): \n', 1e-6*deviator_matrix)

Principal deviator stresses (MPa):  [599999999.9999995, -199999999.9999988, -400000000.0000006]
Dilatational matrix (MPa): 
 [[400.   0.   0.]
 [  0. 400.   0.]
 [  0.   0. 400.]]
Deviator matrix (MPa): 
 [[-300.  100.    0.]
 [ 100. -300.    0.]
 [   0.    0.  600.]]


----------------------------------------------------------

# Utilities

In [118]:
def input_check_Nx1(x):
    """
    Check x to be of dimension Nx1 and reshape it as a 1-D array
    Adhika Lie
    """
    x = np.atleast_1d(x)
    theSize = np.shape(x)

    if(len(theSize) > 1):
        # 1. Input must be of size N x 1
        if ((theSize[0] != 1) & (theSize[1] != 1)):
            raise ValueError('Not an N x 1 array')
        # 2. Make it into a 1-D array
        x = x.reshape(_np.size(x))
    elif (theSize[0] == 1):
        x = x[0]

    return x, np.size(x)

In [116]:
def angle2dcm(rotAngle1, rotAngle2, rotAngle3, input_unit='deg',
              rotation_sequence='ZYX', output_type='ndarray'):
    """
    This function converts Euler Angle into Direction Cosine Matrix (DCM).
    The DCM is described by three sucessive rotation rotAngle1, rotAngle2, and
    rotAngle3 about the axes described by the rotation_sequence.
    The default rotation_sequence='ZYX' is the aerospace sequence and rotAngle1
    is the yaw angle, rotAngle2 is the pitch angle, and rotAngle3 is the roll
    angle. In this case DCM transforms a vector from the locally level
    coordinate frame (i.e. the NED frame) to the body frame.
    This function can batch process a series of rotations (e.g., time series
    of Euler angles).
    Parameters
    ----------
    rotAngle1, rotAngle2, rotAngle3 : angles {(N,), (N,1), or (1,N)}
            They are a sequence of angles about successive axes described by
            rotation_sequence.
    input_unit : {'rad', 'deg'}, optional
            Rotation angles. Default is 'rad'.
    rotation_sequence : {'ZYX'}, optional
            Rotation sequences. Default is 'ZYX'.
    output_type : {'ndarray','matrix'}, optional
            Output type. Default is 'ndarray'.
    Returns
    --------
    C : {3x3} Direction Cosine Matrix
    Notes
    -----
    Programmer:    Adhika Lie
    Created:    	 May 03, 2011
    Last Modified: January 12, 2016
    """
    rotAngle1, N1 = input_check_Nx1(rotAngle1)
    rotAngle2, N2 = input_check_Nx1(rotAngle2)
    rotAngle3, N3 = input_check_Nx1(rotAngle3)

    if(N1 != N2 or N1 != N3):
        raise ValueError('Inputs are not of same dimensions')
    if(N1 > 1 and output_type != 'ndarray'):
        raise ValueError('Matrix output requires scalar inputs')

    R3 = np.zeros((N1, 3, 3))
    R2 = np.zeros((N1, 3, 3))
    R1 = np.zeros((N1, 3, 3))

    if(input_unit == 'deg'):
        rotAngle1 = np.deg2rad(rotAngle1)
        rotAngle2 = np.deg2rad(rotAngle2)
        rotAngle3 = np.deg2rad(rotAngle3)

    R3[:, 2, 2] = 1.0
    R3[:, 0, 0] = np.cos(rotAngle1)
    R3[:, 0, 1] = np.sin(rotAngle1)
    R3[:, 1, 0] = -np.sin(rotAngle1)
    R3[:, 1, 1] = np.cos(rotAngle1)

    R2[:, 1, 1] = 1.0
    R2[:, 0, 0] = np.cos(rotAngle2)
    R2[:, 0, 2] = -np.sin(rotAngle2)
    R2[:, 2, 0] = np.sin(rotAngle2)
    R2[:, 2, 2] = np.cos(rotAngle2)

    R1[:, 0, 0] = 1.0
    R1[:, 1, 1] = np.cos(rotAngle3)
    R1[:, 1, 2] = np.sin(rotAngle3)
    R1[:, 2, 1] = -np.sin(rotAngle3)
    R1[:, 2, 2] = np.cos(rotAngle3)

    if rotation_sequence == 'ZYX':
        try:
            # Equivalent to C = R1.dot(R2.dot(R3)) for each of N inputs but
            # implemented efficiently in C extension
            C = np.einsum('nij, njk, nkm -> nim', R1, R2, R3)
        except AttributeError:
            # Older NumPy without einsum
            C = np.zeros((N1, 3, 3))
            for i, (R1, R2, R3) in enumerate(zip(R1, R2, R3)):
                C[i] = R1.dot(R2.dot(R3))
    else:
        raise NotImplementedError('Rotation sequences other than ZYX are not currently implemented')

    if(N1 == 1):
        C = C[0]
    if(output_type == 'matrix'):
        C = np.matrix(C)

    return C